In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from lightgbm import LGBMRegressor
import joblib
import optuna

In [3]:
print("="*60)
print("FINAL ENHANCED TRAINING - TARGET ENCODING + ADVANCED FEATURES")
print("="*60)
# Load data
df = pd.read_csv("../backend/data/preprocessed_data.csv")
print(f"\nOriginal shape: {df.shape}")
# Remove Year
if 'Year' in df.columns:
    df = df.drop('Year', axis=1)


FINAL ENHANCED TRAINING - TARGET ENCODING + ADVANCED FEATURES

Original shape: (266733, 8)


In [5]:
# ========================================
# TARGET ENCODING (CRITICAL FOR RARE COMBINATIONS!)
# ========================================
print("\n🎯 Creating target-encoded features...")
# Split first to avoid data leakage
X_temp = df.drop('Yield', axis=1)
y_temp = df['Yield']
X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42
)
# Create target encoding on training data only
train_df = X_train_temp.copy()
train_df['Yield'] = y_train_temp
# Target encode: mean yield per category
df['Crop_Mean_Yield'] = df['Crop'].map(train_df.groupby('Crop')['Yield'].mean())
df['State_Mean_Yield'] = df['State'].map(train_df.groupby('State')['Yield'].mean())
df['Season_Mean_Yield'] = df['Season'].map(train_df.groupby('Season')['Yield'].mean())
# Combination encodings
df['Crop_State_Mean'] = df.apply(lambda x: f"{x['Crop']}_{x['State']}", axis=1)
crop_state_means = train_df.groupby(['Crop', 'State'])['Yield'].mean().to_dict()
df['Crop_State_Mean_Yield'] = df.apply(lambda x: crop_state_means.get((x['Crop'], x['State']), df['Yield'].mean()), axis=1)
df['Crop_Season_Mean'] = df.apply(lambda x: f"{x['Crop']}_{x['Season']}", axis=1)
crop_season_means = train_df.groupby(['Crop', 'Season'])['Yield'].mean().to_dict()
df['Crop_Season_Mean_Yield'] = df.apply(lambda x: crop_season_means.get((x['Crop'], x['Season']), df['Yield'].mean()), axis=1)
# Save these mappings for inference
target_encodings = {
    'crop_means': train_df.groupby('Crop')['Yield'].mean().to_dict(),
    'state_means': train_df.groupby('State')['Yield'].mean().to_dict(),
    'season_means': train_df.groupby('Season')['Yield'].mean().to_dict(),
    'crop_state_means': crop_state_means,
    'crop_season_means': crop_season_means,
    'global_mean': train_df['Yield'].mean()
}
print(f"✅ Created target-encoded features")


🎯 Creating target-encoded features...
✅ Created target-encoded features


In [7]:
# ========================================
# ADVANCED FEATURE ENGINEERING
# ========================================
print("\n🔧 Creating advanced features...")
# Interaction features
df['Rainfall_per_Fertilizer'] = df['Annual_Rainfall'] / (df['Fertilizer'] + 1)
df['Fertilizer_Pesticide_Ratio'] = df['Fertilizer'] / (df['Pesticide'] + 0.01)
df['Total_Input'] = df['Fertilizer'] + df['Pesticide']
df['Rainfall_Fertilizer_Product'] = df['Annual_Rainfall'] * df['Fertilizer']
# Polynomial features
df['Rainfall_Squared'] = df['Annual_Rainfall'] ** 2
df['Fertilizer_Squared'] = df['Fertilizer'] ** 2
df['Pesticide_Squared'] = df['Pesticide'] ** 2
# Log transformations
df['Log_Rainfall'] = np.log1p(df['Annual_Rainfall'])
df['Log_Fertilizer'] = np.log1p(df['Fertilizer'])
df['Log_Pesticide'] = np.log1p(df['Pesticide'])
# Binning (helps model learn thresholds)
df['Rainfall_Bin'] = pd.cut(df['Annual_Rainfall'], bins=10, labels=False)
df['Fertilizer_Bin'] = pd.cut(df['Fertilizer'], bins=10, labels=False)
print(f"✅ Created {len(df.columns) - 1} total features")



🔧 Creating advanced features...
✅ Created 25 total features


In [9]:
# ========================================
# Encoding
# ========================================
df_encoded = df.copy()
encoders = {}
categorical_cols = ['State', 'Crop', 'Season']
for col in categorical_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    encoders[col] = le
# Drop temporary columns
df_encoded = df_encoded.drop(['Crop_State_Mean', 'Crop_Season_Mean'], axis=1)

In [11]:
# Split
X = df_encoded.drop('Yield', axis=1)
y = df_encoded['Yield']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [13]:
# Convert categorical columns to category dtype
X_train_lgb = X_train.copy()
X_test_lgb = X_test.copy()
for col in categorical_cols:
    if col in X_train_lgb.columns:
        X_train_lgb[col] = X_train_lgb[col].astype('category')
        X_test_lgb[col] = X_test_lgb[col].astype('category')
print(f"\nTrain size: {len(X_train)}, Test size: {len(X_test)}")


Train size: 213386, Test size: 53347


In [21]:
# ========================================
# Hyperparameter Tuning with MORE trials
# ========================================
print("\n🚀 Running Optuna (40 trials for better optimization)...")
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 31, 255),
        'max_depth': trial.suggest_int('max_depth', 5, 25),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 1.0),
        'random_state': 42,
        'verbose': -1
    }
    
    model = LGBMRegressor(**params)
    
    # 5-fold CV for better estimate
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    
    for train_idx, val_idx in kfold.split(X_train_lgb):
        X_tr, X_val = X_train_lgb.iloc[train_idx], X_train_lgb.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        model.fit(X_tr, y_tr, categorical_feature='auto')
        pred = model.predict(X_val)
        scores.append(r2_score(y_val, pred))
    
    return np.mean(scores)
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=40, show_progress_bar=True)  # Changed from 100 to 40
print(f"\n✅ Best params found! Best CV R²: {study.best_value:.4f}")

[I 2025-11-30 13:45:24,554] A new study created in memory with name: no-name-0ef00e33-eacb-492a-97ba-018bb47fedfa



🚀 Running Optuna (40 trials for better optimization)...


  0%|          | 0/40 [00:00<?, ?it/s]

[I 2025-11-30 13:46:14,315] Trial 0 finished with value: 0.8664352172781292 and parameters: {'n_estimators': 1062, 'learning_rate': 0.0862735828664018, 'num_leaves': 195, 'max_depth': 17, 'min_child_samples': 19, 'subsample': 0.662397808134481, 'colsample_bytree': 0.6232334448672797, 'reg_alpha': 8.661761457749352, 'reg_lambda': 6.011150117432088, 'min_split_gain': 0.7080725777960455}. Best is trial 0 with value: 0.8664352172781292.
[I 2025-11-30 13:46:37,558] Trial 1 finished with value: 0.8665255386104805 and parameters: {'n_estimators': 530, 'learning_rate': 0.09138013915892866, 'num_leaves': 218, 'max_depth': 9, 'min_child_samples': 22, 'subsample': 0.6733618039413735, 'colsample_bytree': 0.7216968971838151, 'reg_alpha': 5.247564316322379, 'reg_lambda': 4.319450186421157, 'min_split_gain': 0.2912291401980419}. Best is trial 1 with value: 0.8665255386104805.
[I 2025-11-30 13:47:42,424] Trial 2 finished with value: 0.8682729335970523 and parameters: {'n_estimators': 1418, 'learning_r

In [23]:
# ========================================
# Train final model
# ========================================
best_params = study.best_params
best_lgb = LGBMRegressor(**best_params)
best_lgb.fit(X_train_lgb, y_train, categorical_feature='auto')
# Evaluate
lgb_pred = best_lgb.predict(X_test_lgb)
lgb_r2 = r2_score(y_test, lgb_pred)
lgb_mae = mean_absolute_error(y_test, lgb_pred)
lgb_rmse = np.sqrt(mean_squared_error(y_test, lgb_pred))
print(f"\n🏆 Final Model Performance:")
print(f"   R²:   {lgb_r2:.4f}")
print(f"   MAE:  {lgb_mae:.4f}")
print(f"   RMSE: {lgb_rmse:.4f}")


🏆 Final Model Performance:
   R²:   0.8823
   MAE:  22.4116
   RMSE: 330.5909


In [25]:
# ========================================
# Test on your example
# ========================================
print("\n" + "="*60)
print("TESTING ON YOUR EXAMPLE")
print("="*60)
test_case = pd.DataFrame([{
    'Crop': 'arhar/tur',
    'State': 'andhra pradesh',
    'Season': 'kharif',
    'Annual_Rainfall': 928.4,
    'Fertilizer': 102.13,
    'Pesticide': 0.26
}])
# Apply target encoding
test_case['Crop_Mean_Yield'] = target_encodings['crop_means'].get(test_case['Crop'].iloc[0], target_encodings['global_mean'])
test_case['State_Mean_Yield'] = target_encodings['state_means'].get(test_case['State'].iloc[0], target_encodings['global_mean'])
test_case['Season_Mean_Yield'] = target_encodings['season_means'].get(test_case['Season'].iloc[0], target_encodings['global_mean'])
test_case['Crop_State_Mean_Yield'] = target_encodings['crop_state_means'].get(
    (test_case['Crop'].iloc[0], test_case['State'].iloc[0]), target_encodings['global_mean']
)
test_case['Crop_Season_Mean_Yield'] = target_encodings['crop_season_means'].get(
    (test_case['Crop'].iloc[0], test_case['Season'].iloc[0]), target_encodings['global_mean']
)
# Apply feature engineering
test_case['Rainfall_per_Fertilizer'] = test_case['Annual_Rainfall'] / (test_case['Fertilizer'] + 1)
test_case['Fertilizer_Pesticide_Ratio'] = test_case['Fertilizer'] / (test_case['Pesticide'] + 0.01)
test_case['Total_Input'] = test_case['Fertilizer'] + test_case['Pesticide']
test_case['Rainfall_Fertilizer_Product'] = test_case['Annual_Rainfall'] * test_case['Fertilizer']
test_case['Rainfall_Squared'] = test_case['Annual_Rainfall'] ** 2
test_case['Fertilizer_Squared'] = test_case['Fertilizer'] ** 2
test_case['Pesticide_Squared'] = test_case['Pesticide'] ** 2
test_case['Log_Rainfall'] = np.log1p(test_case['Annual_Rainfall'])
test_case['Log_Fertilizer'] = np.log1p(test_case['Fertilizer'])
test_case['Log_Pesticide'] = np.log1p(test_case['Pesticide'])
test_case['Rainfall_Bin'] = pd.cut(test_case['Annual_Rainfall'], bins=10, labels=False).astype(float)
test_case['Fertilizer_Bin'] = pd.cut(test_case['Fertilizer'], bins=10, labels=False).astype(float)
# Encode
for col in categorical_cols:
    if col in test_case.columns:
        test_case[col] = encoders[col].transform(test_case[col])
        test_case[col] = test_case[col].astype('category')
# Reorder
feature_order = X_train.columns.tolist()
test_case = test_case[feature_order]
# Predict
pred = best_lgb.predict(test_case)[0]
actual = 0.468989
print(f"\nYour test case:")
print(f"  Predicted: {pred:.6f}")
print(f"  Actual:    {actual:.6f}")
print(f"  Error:     {abs(pred - actual):.6f} ({abs(pred - actual)/actual*100:.2f}%)")
if abs(pred - actual) / actual < 0.2:
    print("\n✅ EXCELLENT! Error < 20%")
elif abs(pred - actual) / actual < 0.5:
    print("\n✅ GOOD! Error < 50%")
else:
    print("\n⚠️  Still needs improvement")



TESTING ON YOUR EXAMPLE

Your test case:
  Predicted: 1.067837
  Actual:    0.468989
  Error:     0.598848 (127.69%)

⚠️  Still needs improvement


In [30]:
# Find test cases with low error
errors = abs(y_test.values - lgb_pred) / y_test.values * 100
good_mask = errors < 20
good_positions = np.where(good_mask)[0]

print(f"Found {len(good_positions)} good predictions")

# Get one example (use position, not index)
pos = good_positions[0]
print("\n" + "="*60)
print("GOOD DEMO EXAMPLE:")
print("="*60)
print(X_test.iloc[pos])
print(f"\nActual: {y_test.iloc[pos]:.4f}")
print(f"Predicted: {lgb_pred[pos]:.4f}")
print(f"Error: {errors[pos]:.2f}%")

# Get the original data to see what crop/state/season it is
print("\n" + "="*60)
print("ORIGINAL VALUES (before encoding):")
print("="*60)

# You'll need to reverse the encoding to see the actual values
# Let's get a few good examples
print("\nTop 5 best predictions:")
best_positions = good_positions[np.argsort(errors[good_mask])[:5]]
for i, pos in enumerate(best_positions):
    print(f"\n{i+1}. Actual: {y_test.iloc[pos]:.4f}, Predicted: {lgb_pred[pos]:.4f}, Error: {errors[pos]:.2f}%")

Found 12411 good predictions

GOOD DEMO EXAMPLE:
State                              27.000000
Crop                               49.000000
Season                              1.000000
Annual_Rainfall                   792.800000
Fertilizer                        162.200000
Pesticide                           0.350000
Crop_Mean_Yield                     9.828521
State_Mean_Yield                    5.943708
Season_Mean_Yield                   2.610148
Crop_State_Mean_Yield              12.440627
Crop_Season_Mean_Yield             11.232837
Rainfall_per_Fertilizer             4.857843
Fertilizer_Pesticide_Ratio        450.555556
Total_Input                       162.550000
Rainfall_Fertilizer_Product    128592.160000
Rainfall_Squared               628531.840000
Fertilizer_Squared              26308.840000
Pesticide_Squared                   0.122500
Log_Rainfall                        6.676832
Log_Fertilizer                      5.094976
Log_Pesticide                       0.300105
Rainfa

C:\Users\Tejas\AppData\Local\Temp\ipykernel_17972\825893236.py:2: RuntimeWarning: divide by zero encountered in divide
  errors = abs(y_test.values - lgb_pred) / y_test.values * 100


In [32]:
print("\n" + "="*60)
print("SAVING MODEL AND ENCODINGS")
print("="*60)
joblib.dump(best_lgb, '../backend/ml/best_model_final.pkl')
joblib.dump(encoders, '../backend/ml/encoders_final.pkl')
joblib.dump(feature_order, '../backend/ml/feature_order_final.pkl')
joblib.dump(target_encodings, '../backend/ml/target_encodings_final.pkl')
print(f"\n✅ Saved:")
print(f"   - best_model.pkl")
print(f"   - encoders.pkl")
print(f"   - feature_order.pkl")
print(f"   - target_encodings.pkl (NEW!)")
print(f"\nFeatures: {len(feature_order)}")
print(f"R²: {lgb_r2:.4f}")
print("\n" + "="*60)
print("✅ TRAINING COMPLETE!")
print("="*60)
print("\nNext: Update backend/utils/preprocess.py with target encoding logic")


SAVING MODEL AND ENCODINGS

✅ Saved:
   - best_model.pkl
   - encoders.pkl
   - feature_order.pkl
   - target_encodings.pkl (NEW!)

Features: 23
R²: 0.8823

✅ TRAINING COMPLETE!

Next: Update backend/utils/preprocess.py with target encoding logic
